In [422]:
import openai
import csv
import ast
import collections

In [738]:
prompt="Please summarize the diseases and treatments mentioned.\n\nTo study the safety and efficacy of methylphenidate in children with the dual diagnosis of epilepsy and attention deficit hyperactivity disorder ( ADHD ) .\nDisease: epilepsy ; attention deficit hyperactivity disorder ( ADHD )\nTreatment:  methylphenidate\n\nMethylphenidate is effective in treating children with epilepsy and ADHD and safe in children who are seizure free .\nDisease: epilepsy ; ADHD\nTreatment:  Methylphenidate\n\nAntiplatelet therapy with aspirin and systematic anticoagulation with warfarin reduce cardiovascular morbidity and mortality after myocardial infarction when given alone .\nDisease: cardiovascular morbidity and mortality after myocardial infarction\nTreatment: Antiplatelet therapy with aspirin and systematic anticoagulation with warfarin\n\nLow , fixed-dose warfarin ( 1 mg or 3 mg ) combined with low-dose aspirin ( 80 mg ) in patients who have had myocardial infarction does not provide clinical benefit beyond that achievable with 160 mg aspirin monotherapy .\nDisease:  myocardial infarction\nTreatment: warfarin ( 1 mg or 3 mg ) combined with low-dose aspirin ( 80 mg ) ; aspirin monotherapy\n\nAt 293 sites , we randomly assigned 8803 patients who had had myocardial infarction , treatment with 160 mg aspirin , 3 mg warfarin with 80 mg aspirin , or 1 mg warfarin with 80 mg aspirin .\nDisease: myocardial infarction\nTreatment: 160 mg aspirin ; 3 mg warfarin with 80 mg aspirin ; 1 mg warfarin with 80 mg aspirin"
sents=[]
real_dis=[]
real_treat=[]

with open('BioText_prepared.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            sents.append(row[0])
            real_dis.append(ast.literal_eval(row[1].lower()))
            real_treat.append(ast.literal_eval(row[2].lower()))
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are TEXT, DISEASE, TREATMENT
Processed 830 lines.


In [739]:
fin_pmt=[]

# Prepare all the prompts to send
for sent in sents[5:]:
    fin_pmt.append(prompt + "\n\n" + sent + '\nDisease:')


In [660]:
openai.api_key = "<YOUR_KEY>"
response=[]

for cur_pmt in fin_pmt: #todo
    response.append(openai.Completion.create(
      engine="curie",
      prompt= cur_pmt,
      temperature=0,
      max_tokens=50,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n\n"]
    ))

In [741]:
rsp_dis = []
rsp_treat = []
rsp=[]

#Collect the result from the GPT3 response
for file in response:
    rsp.append(file["choices"][0]['text'].lower())

#Making the strings comparable.
for cur in rsp:
    if cur.startswith(" "):
        cur = cur[1:]

    tmp = cur.splitlines()
    tmp_dis = tmp[0]
    tmp_treat = tmp[1].replace('treatment: ','')

    rsp_dis.append(tmp_dis.split(" ; "))
    rsp_treat.append(tmp_treat.split(" ; "))

[['unstable angina or non-q-wave myocardial infarction'], ['coronary-artery disease'], ['cervical carcinoma'], ['myofascial trigger points of upper trapezius muscle'], ['myofascial trigger point pain'], ['obesity'], ['essential hypertension'], ['echinococcus multilocularis'], ['foot infections in diabetic patients'], ['foot infection'], ['coronary-artery thrombi'], ['acute myocardial infarction'], ['rheumatoid arthritis'], ["parkinson 's disease"], ["parkinson 's disease"], ['primary cataract surgery with pc iol implantation'], ['sore throat'], ['sore throat'], ['sore throat'], ['sore throat'], ['postoperative pain relief'], ['stress urinary incontinence'], ['stress urinary incontinence'], ['stress urinary incontinence'], ['stress urinary incontinence'], ['primary infertility'], ['primary infertility'], ['gunshot wounds to the abdomen'], ['major pulmonary embolism'], ['pulmonary embolism'], ['malignant pleural mesothelioma'], ["parkinson 's disease"], ['non-obstructive azoospermia'], [

In [745]:
### Determines whether the strings are identical or not
def determinator(kk, rsp_t_or_d, real_t_or_d, tmp):
    if collections.Counter(rsp_t_or_d[kk]) != collections.Counter(real_t_or_d[kk]):
        tmp.append(False)
    else:
        tmp.append(True)
    if kk+1 == len(rsp_t_or_d):
        for k in range(len(real_t_or_d) - (kk+1)):
            tmp.append(False)
        return True
    return False

###  Iterates every response to be evaluated
def exact_wording(flag):
    for n in range(len(rsp)): #l > 5
        tmp_t =[]
        tmp_d =[]

        for kk in range(len(real_dis[n+5])):
            if determinator(kk, rsp_dis[n], real_dis[n+5], tmp_d):
                break

        for kk in range(len(real_treat[n+5])):
            if determinator(kk, rsp_treat[n], real_treat[n+5], tmp_t):
                break

        flag.append([tmp_d, tmp_t])

### Evaluates the accuracy comparing each solution with in the response string with the dataset.
def correct_evaluation():
    flag=[]
    exact_wording(flag)

    t_count = 0
    f_count = 0

    for row in flag:
        for set in row:
            for val in set:
                if val:
                    t_count += 1
                else:
                    f_count += 1

    print("Ratio wording (True, False): ", t_count, f_count)
    return t_count/(t_count+f_count)

### Evaluates the accuracy comparing the response string with the dataset string
def correct_solution():
    flag=[]
    exact_wording(flag)
    count = 0

    for row in flag:
        switch = False
        for set in row:
            for val in set:
                if not val:
                    switch=True
        if not switch:
            count+=1

    print("Ratio Solution: ", count, len(flag))
    return count/len(flag)

In [753]:
### Splits treatments and diseases solutions on "or"
def rough(rsp_t_or_d):
    new_rsp = []

    for row in rsp_t_or_d:
        new_row = []
        for val in row:
            for each in val.split(" or "):
                new_row.append(each)
        new_rsp.append(new_row)

    return new_rsp

###  Iterates every response to be evaluated
def rough_wording(flag):
    rough_rsp_dis = rough(rsp_dis)
    rough_rsp_treat = rough(rsp_treat)

    for n in range(len(rsp)):
        tmp_t =[]
        tmp_d =[]

        for kk in range(len(real_dis[n+5])):
            if determinator(kk, rough_rsp_dis[n], real_dis[n+5], tmp_d):
                break

        for kk in range(len(real_treat[n+5])):
            if determinator(kk, rough_rsp_treat[n], real_treat[n+5], tmp_t):
                break

        flag.append([tmp_d, tmp_t])

### Evaluates the accuracy comparing each further post-processed solution with in the response string with the dataset.
def rough_evaluation():
    flag=[]
    rough_wording(flag)

    t_count = 0
    f_count = 0

    for row in flag:
        for set in row:
            for val in set:
                if val:
                    t_count += 1
                else:
                    f_count += 1
    print("Ratio wording (True, False): ", t_count, f_count)
    return t_count/(t_count+f_count)

### Evaluates the accuracy comparing the further post-proccessed response string with the dataset string
def rough_solution():
    flag=[]
    rough_wording(flag)
    count = 0

    for row in flag:
        switch = False
        for set in row:
            for val in set:
                if not val:
                    switch=True
        if not switch:
            count+=1
    print("Ratio Rough Solution: ", count, len(flag))
    return count/len(flag)

In [751]:
print("------------------")
c_s=correct_solution()

print("------------------")
o_e_w=correct_evaluation()

print("------------------")
r_s=rough_solution()

print("------------------")
r_w=rough_evaluation()

------------------
Ratio Solution:  300 824
------------------
Ratio wording (True, False):  1041 798
------------------
Ratio Rough Solution:  301 824
------------------
Ratio wording (True, False):  1047 792


In [750]:
print("Correct_Completion - Evaluation (%): ",c_s*100, "%")

print("Correct_wording - Evaluation (%): ",o_e_w*100, "%")

print("Rough_Completion - Evaluation (%): ",r_s*100, "%")

print("Rough_wording - Evaluation (%): ",r_w*100, "%")

Correct_Completion - Evaluation (%):  36.407766990291265 %
Overall_exact_wording - Evaluation (%):  56.6068515497553 %
Rough_Completion - Evaluation (%):  36.529126213592235 %
Rough_wording - Evaluation (%):  56.933115823817296 %
